# Automatic Summarization of Websites

Student: Karin Brodd

Course w266 section 2

In [69]:
%load_ext autoreload
%autoreload 2
import get_data
import process_data
import summary_baseline
from importlib import reload
from langdetect import detect
from nltk.corpus import stopwords

import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pytextrank
import sys
from toolbox import WordDistribution
from collections import namedtuple
from sumy.parsers.html import HtmlParser
from sumy.nlp.tokenizers import Tokenizer 
from sumy.summarizers.lex_rank import LexRankSummarizer


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
!mkdir data
!mkdir data/paths
!mkdir data/warc

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/paths’: File exists
mkdir: cannot create directory ‘data/warc’: File exists


## Download data

In [266]:
reload(get_data)
warc_paths = get_data.get_warc_paths()
sample = get_data.get_warc_files(warc_paths, 10)
print sample

['crawl-data/CC-MAIN-2017-43/segments/1508187823482.25/warc/CC-MAIN-20171019231858-20171020011858-00601.warc.gz', 'crawl-data/CC-MAIN-2017-43/segments/1508187822625.57/warc/CC-MAIN-20171017234801-20171018014801-00052.warc.gz', 'crawl-data/CC-MAIN-2017-43/segments/1508187825399.73/warc/CC-MAIN-20171022165927-20171022185927-00053.warc.gz', 'crawl-data/CC-MAIN-2017-43/segments/1508187825308.77/warc/CC-MAIN-20171022150946-20171022170946-00441.warc.gz', 'crawl-data/CC-MAIN-2017-43/segments/1508187823360.1/warc/CC-MAIN-20171019175016-20171019195016-00332.warc.gz', 'crawl-data/CC-MAIN-2017-43/segments/1508187828411.81/warc/CC-MAIN-20171024105736-20171024125736-00136.warc.gz', 'crawl-data/CC-MAIN-2017-43/segments/1508187823630.63/warc/CC-MAIN-20171020025810-20171020045810-00399.warc.gz', 'crawl-data/CC-MAIN-2017-43/segments/1508187825812.89/warc/CC-MAIN-20171023073607-20171023093607-00712.warc.gz', 'crawl-data/CC-MAIN-2017-43/segments/1508187823360.1/warc/CC-MAIN-20171019175016-20171019195016-

## Run baseline summary on sites

In [81]:
files_list = [x for x in os.listdir("data/warc") if x.startswith('CC-MAIN')]
url_documents = []
for warc in files_list:
    path = "data/warc/" + warc
    documents, urls = process_data.read_record(path, num_pages=5)
    url_documents += [x for x in zip(urls, documents)]
    
stop_words = set(stopwords.words('english'))
for url, document in url_documents:
    if detect(document) != 'en':
        continue
    doc_distribution = WordDistribution(document, stop_words=stop_words, smoothing=0)
    print(doc_distribution.word_distribution)

{'usagenre': 0.005050505050505051, 'genres': 0.005050505050505051, '20172016201520142013201220112010200920082007200620052004200320022001200019991998199719961995199419931992199119901989198819871986198519841983198219811980197919781977197619751974197319721971': 0.0025252525252525255, 'full': 0.0025252525252525255, 'movieadd': 0.005050505050505051, 'subscribe': 0.007575757575757576, 'woods': 0.005050505050505051, 'actionanimationfantasyhorrormusicalromance': 0.0025252525252525255, 'enter': 0.0025252525252525255, 'verna': 0.005050505050505051, 'night': 0.0025252525252525255, 'democratic': 0.0025252525252525255, 'phipps': 0.0025252525252525255, 'fill': 0.0025252525252525255, 'trailer': 0.0025252525252525255, 'allafghanistanalbaniaalgeriaandorraangolaargentinaarmeniaarubaaustraliaaustriabahamasbangladeshbelarusbelgiumbelizebermudaboliviabosnia': 0.0025252525252525255, 'lankaswedenswitzerlandsyriataiwantanzaniathailandthe': 0.0025252525252525255, 'away': 0.005050505050505051, 'home': 0.0025252

{'lbgŕɓi': 0.0005534034311012728, 'tłłăt~ȏ҂鏤ނəoi': 0.0005534034311012728, '22': 0.0005534034311012728, '//tinyurl.com/fl8kzt': 0.0005534034311012728, '`imȃƒi': 0.02213613724405091, 'nvōp': 0.0005534034311012728, 'x': 0.0005534034311012728, 'xfprice': 0.0005534034311012728, '//ininin.info/817`gюs10nõc^': 0.0005534034311012728, '̓r`http': 0.0005534034311012728, '00:43': 0.0005534034311012728, 'lɍȃu҂vlbgɓ\\i': 0.0005534034311012728, 'toeic900œzr': 0.0005534034311012728, 'ǘlљ': 0.0005534034311012728, 'iyiڍׁzt': 0.0005534034311012728, 'nvxv': 0.0011068068622025456, '//ininin.info/818150y': 0.0005534034311012728, 'v80̏ނ̒t': 0.0005534034311012728, '//ininin.info/819\U0004f90dނ\U000d57d1܁': 0.0005534034311012728, ':': 0.11621472053126729, 'ʂ̎wt': 0.0005534034311012728, '//tinyurl.com/fmog7oc': 0.0005534034311012728, 'ȃ̐liɕςc': 0.0011068068622025456, 'o^`ded': 0.0005534034311012728, 'ǂăȃ\U00042f81hǂł҂i': 0.0005534034311012728, '09:02': 0.0005534034311012728, 'wlbgi': 0.0005534034311012728, 'mȃ

{'2012it': 0.0013012361743656475, 'light': 0.003903708523096942, 'end': 0.0006506180871828237, '..replydeleteanonymoussaturday': 0.0006506180871828237, 'stopped': 0.0006506180871828237, '22': 0.0013012361743656475, 'states': 0.0006506180871828237, '2002': 0.0006506180871828237, 'board': 0.001951854261548471, 'department': 0.0032530904359141183, 'ignorant': 0.0006506180871828237, 'december': 0.001951854261548471, 'listening': 0.0006506180871828237, 'numbered': 0.0006506180871828237, '.replydeleteadd': 0.0006506180871828237, 'accidents': 0.004554326610279766, 'factual': 0.0013012361743656475, 'maria': 0.0006506180871828237, 'wboe': 0.0006506180871828237, 'factually': 0.0006506180871828237, 'ave.': 0.0032530904359141183, 'wife': 0.0006506180871828237, 'resulted': 0.0006506180871828237, 'dear': 0.0006506180871828237, 'misbehavior': 0.0006506180871828237, 'another': 0.0013012361743656475, 'stealing': 0.0006506180871828237, 'reading': 0.0013012361743656475, 'lying': 0.0006506180871828237, 'm

{'hot': 0.001953125, 'fell': 0.001953125, 'even': 0.001953125, 'showed': 0.001953125, 'salt': 0.001953125, 'subscribe': 0.001953125, 'wonderful': 0.001953125, 'oregano': 0.001953125, 'versatile': 0.001953125, 'never': 0.00390625, 'interesting': 0.001953125, 'december': 0.00390625, 'designof': 0.001953125, 'traditional': 0.00390625, 'dried': 0.00390625, 'bone': 0.001953125, 'familymay': 0.00390625, 'applied': 0.001953125, 'may': 0.00390625, 'replydeletebell': 0.001953125, 'dinner': 0.001953125, '9:25': 0.001953125, 'awesome': 0.001953125, 'home': 0.001953125, 'family': 0.013671875, 'bye': 0.001953125, 'love': 0.001953125, 'smoke': 0.001953125, 'turkey': 0.033203125, '|': 0.001953125, 'wife': 0.001953125, 'sillitoe': 0.001953125, '1': 0.01953125, 'reading': 0.001953125, 'inside': 0.00390625, 'buckshot': 0.00390625, 'best': 0.005859375, 'widget': 0.001953125, 'amthat': 0.001953125, 'replydeletejensen': 0.001953125, 'shelves': 0.001953125, 'march': 0.001953125, ':': 0.005859375, 'pmthe': 0

{'ӱ': 0.0011350737797956867, 'isblackvisitor': 0.0011350737797956867, 'ɨһɨ': 0.004540295119182747, 'permalink': 0.0011350737797956867, 'lofterѳӡ20ƭн': 0.0011350737797956867, 'first_option': 0.0011350737797956867, 'blogdetail.preblogpermalink': 0.0011350737797956867, 'ȵ': 0.0011350737797956867, 'x': 0.02724177071509648, '001919dyd־': 0.0011350737797956867, 'votetooption': 0.0011350737797956867, '|escape': 0.011350737797956867, '--': 0.01021566401816118, '·ȧ': 0.004540295119182747, '$': 0.03178206583427923, 'great260': 0.0011350737797956867, 'ײ': 0.0011350737797956867, '/quote': 0.0022701475595913734, 'loftpermalink': 0.0011350737797956867, 'fn1': 0.0022701475595913734, '|': 0.007945516458569807, 'ر': 0.00340522133938706, 'һҹ': 0.0011350737797956867, 'x.publishtime': 0.0011350737797956867, 'visitorprovince': 0.0011350737797956867, 'ͷ': 0.0011350737797956867, '|ٱ': 0.0011350737797956867, 'ҷҷҵӳκʱκʱ\U000a3a30ĸ翪ľ죩сˡ¼ֻʱ1928-1937ʮʱ伢ķ650\U000a3b3eˡ·ұʬǳ¶족ô͵ǡǧ\U000a1c63ƽÿǧˣġǧˉvѩںδó֤ݳƣ30ġҥ¡ǵǡ': 0.

{'end': 0.00045187528242205153, '%': 0.00045187528242205153, 'research…': 0.00022593764121102577, 'split-canoe': 0.00022593764121102577, 'advocated': 0.0006778129236330773, 'industries': 0.00045187528242205153, 'chatting': 0.00022593764121102577, 'efforts': 0.00022593764121102577, 'parallel': 0.00022593764121102577, '10:24': 0.00022593764121102577, 'trace': 0.00045187528242205153, 'bright': 0.00022593764121102577, 'stuck': 0.00022593764121102577, 'fiasco': 0.00022593764121102577, 'hyperlocal': 0.00022593764121102577, 'culturelonny': 0.00022593764121102577, 'quotes': 0.00022593764121102577, 'audience': 0.0009037505648441031, 'rape': 0.0006778129236330773, 'appropriate': 0.00022593764121102577, 'far': 0.00022593764121102577, '15.': 0.00022593764121102577, 'meltdown': 0.00022593764121102577, 'ways': 0.00022593764121102577, 'holman': 0.0006778129236330773, 'tracing': 0.00022593764121102577, 'conspiracy': 0.00022593764121102577, 'interface': 0.00045187528242205153, 'response': 0.00067781292

{'շƿ10ԫ': 0.004694835680751174, '%': 0.004694835680751174, '//wwwuuggclub.sh-wanqiu.cn/http': 0.004694835680751174, '»': 0.004694835680751174, 'с\u05f94ù˾ҵٻϊ': 0.004694835680751174, '//wwwxmsyhycom.zaoxuewuye.cn/http': 0.004694835680751174, '1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|31|32|33|34|35|36|37|38|39|40|41|42|43|44|45|46|47|48|49|50|51|52|53|54|55|56|57|58|59|60|61|62|63|64|65|66|67|68|69|70|71|72|73|74|75|76|77|78|79|80|81|82|83|84|85|86|87|88|89|90|91|92|93|94|95|96|97|98|99|100|': 0.004694835680751174, '//wwwxh009009com.yctsexy.cn/http': 0.004694835680751174, '//wwwy35com.gzjiawei86.cn/http': 0.004694835680751174, '//wwwxnfbookcom.ys168169.cn/http': 0.004694835680751174, '//wwwxhjc0002com.gzjiawei86.cn/http': 0.004694835680751174, '¼ϰƽҫ쵼ɲ': 0.004694835680751174, '//wwwxiumi749com.mateing.cn/http': 0.004694835680751174, '//wwwxam03com.st-huahai.cn/http': 0.004694835680751174, 'ۣθӽʽѵڲ廳ϊ¸ҿ\U00061c3d¡ƿǡؼɡʶ71ƽѿ': 0.004694835680751174, 'ʒů':

In [80]:
files_list = [x for x in os.listdir("data/warc") if x.startswith('CC-MAIN')]
url_documents = []
stop_words = set(stopwords.words('english'))
for warc in files_list:
    path = "data/warc/" + warc
    documents, urls = process_data.read_record(path, num_pages=5)
    url_documents += [x for x in zip(urls, documents)]

for url, document in url_documents:
    if detect(document) != 'en':
        continue
        
    sent_tokens = process_data.parse_sentences(document)
    #word_counts = summary_baseline.calculate_frequencies(sent_tokens)
    doc_distribution = WordDistribution(document, stop_words=stop_words, smoothing=0)
    scores = summary_baseline.score_sentences(sent_tokens, doc_distribution.word_distribution)

    print('=' * 50)
    print("Domain: " + url)
    summary = ""
    for i in sorted(scores, key=scores.get, reverse=True)[:5]:
        summary += " ".join(sent_tokens[i]) + " "
    print(summary)
    doc_distribution = WordDistribution(document, stop_words=stop_words)
    summary_distribution = WordDistribution(summary, vocabulary=doc_distribution.vocabulary)
    print(summary_distribution.kl_divergence(doc_distribution))
    

Domain: http://0123movies.com/actors/Don+Barclay.html
for everybody , everywhere , everydevice , and everything ; ) remember me forgot password ? 123movies 123movies free free movies free movies online cinema movies watch series free series free online trailer : report : please enter the code submit download subtitles download member login area watch hd movies online for free and download the latest movies . you are welcome when becoming members of the site , you could use the full range of functions and enjoy the most exciting films . login login in with facebook login in with twitter login in with google not a member ? submit back to login 
0.214348934315
Domain: http://024.clayton.k12.ga.us/cms/One.aspx?portalId=64029&pageId=4051055&objectId.271472=4051072&contextId.271472=4051056&parentId.271472=4051064&action.271472=viewImage2
zion elementarymt . ptsa meeting - october 2016 - kendrick middle school skip navigationselect a schooldistrictccpselementaryanderson elementaryarnold eleme

Domain: http://0166244.blogspot.com/2014/
but that ’ s not what i am doing here . 18 . 21 . ( link . ( link . 
0.783988545226
Domain: http://02e5b47.netsolhost.com/images/Woodstock_Charrette_final_report.pdf
? ? ? ? ! 
3.56936322053
Domain: http://0597rcw.xingdewei.cn/my/space/kk4ed.html
ӧÿ ˽_ö_ֲٿͨ ѷ ֲѧ ͥ ԰ ֲͼƭ ֲ¼ ѷ ֲѧ ͥ ҳ > ſѷ > б ӧÿ ˽_ö2017-10-24 05:06:02 ӧÿ ˽µ²ʒƴĺѷҵͻʾʼδȡĳ ! ʹݱ淢 ӧÿ ˽ס÷沿ŷ򽫲ȡͳܡӹҳȴʩʵؿ롢ﻧŀˮϲҵ˽ⱦߺҵر˳г޵޹˾޵оччʡִͷˮƽγõȫաˡȫу԰χ լ˾լ߱ʾʊԥĳ ôõʱ ϊһϵ觶ηϲաͻǰֹϲ罨ɹʡʳʒҩʒලҫբϸʒӫ߽һ鴦 һͬĺڱ۾ » 3.7ٷֵͨqq΢빺աϵ˻齫ƶźżͼƭϣʵʱشϸչʱɳ41ѷձ ؼ ʱż ۣθӽʽѵڲ廳ϊ¸ҿ񡰽¡ƿǡؼɡʶ71ƽѿ 51ӫȫƿ3¡ʹñյĺʱܱǿ¡ǳʹڡäա˽ϊ˻ίίȿҵʹѡõȵҫ֪ʶȩҳ֮խխӡúϵĺ δ۾ɲļȳɸⱦȳӣҫǰãҧļբգѫȼȫը÷ѵսȡȫ˴ίաίաĺھ֯ڻݰ޸ļ㱨ʱũҵշ;ɺӡʱտйϰƽϯڽݿ˷ʵıɫ·ҵуͣɴٿ2015˹ûгσӿ̬ѳ1.1ķ㷨ƶեֵ˰ǰͷֵҽ˰եֿѳָġβݼ͡һǿܵερҫ426ѡֽ潭ϊȫ˵簲ȫˮƽ ÷ƿվֺ̨ г屣ı ߽йƺŀб ʱ̵űʾ տ˿ƭҵ ƻϵͳװ̳̽ӻ׼30ͷţ1000ֻ10 ־һͬĺڱ۾ ͧ ӧüȥ աȩ嵥嵥嵥ġƻ˸ҫ ݼйͨr1߹̽ͷ2 . 2ԫ ͵绰硰ллӻᶥ족 ʪʹùй ʒƥů426 ů б ʒů ˫ҳǹʵ ʈѷĺ ʼ챨ձڴ ձרձͷсǰȥϸ business casual ѷҵͻʾʼδȡեϊͳ ʒƥůע淶 32һϸ ϲŷ򽫲ȡͳܡӹҳȴʩ ȷһĳ ! ·庸ȷηֹ ׿ƽϸéĺа취涨20ʹǳ20һ˵ŀ ˿֯ȥ˾i̇ïٵȸٹ· ʲôչͼ 齭߷ȫ21с200ũ ΢ǿһ廯 ʹƶʵز鿴ũ԰ľ״ շƿ10ԫ ¼ϰƽҫ쵼ɲ ֻ԰ϣͬ־ѧϰ photoshop ľеίֲ١걾

In [68]:
RankedLexeme = namedtuple('RankedLexeme', 'text, rank, ids, pos, count')
files_list = [x for x in os.listdir("data/warc") if x.startswith('CC-MAIN')]
i = 1
for warc in files_list:
    path = "data/warc/" + warc
    documents, urls = process_data.read_record(path, num_pages=5)
    url_documents = zip(urls, documents)
    i = 1
    for url, doc in url_documents:
        if detect(doc) != 'en':
            continue
        # prepare document dictionary for parsing
        doc_iter = (json for json in [{"id": url, "text": doc}])
        # parse document
        parsed_docs = [x._asdict() for x in pytextrank.parse_doc(doc_iter)]
        # run text rank
        graph, ranks =  pytextrank.text_rank(iter(parsed_docs))
        # normalize key phrases
        normalized_key_phrases = [x._asdict() for x in pytextrank.normalize_key_phrases(iter(parsed_docs), ranks)]
        
        kernel = pytextrank.rank_kernel(iter(normalized_key_phrases))
        top_sentences = [x._asdict() for x in pytextrank.top_sentences(kernel, iter(parsed_docs))]
        
        phrases = " ".join(set([p for p in pytextrank.limit_keyphrases(normalized_key_phrases, phrase_limit=12)]))
        sent_iter = sorted(pytextrank.limit_sentences(iter(top_sentences), word_limit=150), key=lambda x: x[1])
        
        s = []
        for sent_text, idx in sent_iter:
            s.append(pytextrank.make_sentence(sent_text))
        graf_text = " ".join(s)
        
        doc_distribution = WordDistribution(doc)
        summary_distribution = WordDistribution(graf_text, vocabulary=doc_distribution.vocabulary)
        print("=" * 50)
        print("url: " + url)
        print("**keywords**: {}".format(phrases))
        print(summary_distribution.kl_divergence(doc_distribution))
        i += 1
        

url: http://0123movies.com/actors/Don+Barclay.html
**keywords**: watch movies luis van rooten macedoniaromaniarussiasamoasaudi arabiaserbiasingaporeslovakiasloveniasouth africasouth koreasoviet unionspainsri top movies republicczechoslovakiadenmarkdominican republicecuadoregyptestoniaethiopiafederal republic barclay rhoda williams top imdb list movies barclaywatch movieadd hd movies release year movies
0.232180165491
url: http://024.clayton.k12.ga.us/cms/One.aspx?portalId=64029&pageId=4051055&objectId.271472=4051072&contextId.271472=4051056&parentId.271472=4051064&action.271472=viewImage2
**keywords**: rallynational school lunch son ribbon weekpresidential election resultsmother healthy ga awardsthank you!career daystudent council highlovejoy highmartha ellen stilwell school september students elementaryeast clayton school monthseptember students honor falcons!ptsa meetingkms students august students bowl competesswimmers students
0.15157027578
url: http://0101010.seesaa.net/article/14

In [64]:
files_list = [x for x in os.listdir("data/warc") if x.startswith('CC-MAIN')]
url_documents = []
for warc in files_list:
    path = "data/warc/" + warc
    documents, urls = process_data.read_record(path, num_pages=5)
    for url, document in zip(urls, documents):
        if detect(document) != 'en':
            continue
        parser = HtmlParser(document, Tokenizer('english'), url)
        summarizer = LexRankSummarizer()

        summary = summarizer(parser.document, 5)
        
        print("=" * 30)
        print("Document: " + url)
        for sentence in summary:
            print(sentence)
            print('-' * 30)
        doc_distribution = WordDistribution(document)
        summary_distribution = WordDistribution(' '.join(str(summary)), vocabulary=doc_distribution.vocabulary)
        print(summary_distribution.kl_divergence(doc_distribution))

Document: http://0123movies.com/actors/Don+Barclay.html
watch don barclay's movies, list movies of don barclay online free 2017, best movies don barclay collection
night mode
watch movies free online - 123movies.com
home
genres
actionadventureanimationbiographycomedycrimedocumentarydramafamilyfantasyfilm-noirhistoryhorrormusicmusicalmysterynewsromancesci-fishortsportthrillerwarwestern
country
australiacanadachinafrancegermanyhong kongindiaitalyjapanrussiasouth koreaswedenthailandukusa
featured
movies
tv-series
top imdb
a-z list
request
login
don barclay
filter
order:
default
new update
most viewed
release year
movies name
imdb
genre: allactionadultadventureanimationbiographycomedycrimedocumentarydramafamilyfantasyfilm-noirgame-showhistoryhorrormusicmusicalmysterynewsreality-tvromancesci-fishortsporttalk-showthrillerwarwesterncountry: allafghanistanalbaniaalgeriaandorraangolaargentinaarmeniaarubaaustraliaaustriabahamasbangladeshbelarusbelgiumbelizebermudaboliviabosnia and herzegovinabot

Document: http://0101010.seesaa.net/article/14627879.html
tinppiłj
excerpt: cɏn߂̒mb8naό200~iȓmbƃ?
------------------------------
colbgi[nvxzp]_price : 1,55...
weblog: oꃊt[`imȃƒi[nv̏펯e펯i
tracked: 2006-04-25 08:32
nɂזꂸɏ𓾂@
excerpt: nɂזꂸɏ𓾂@yiڍׁzt[i[nvőucniw...
weblog: oꃊt[`imȃƒi[nv̏펯e펯i
tracked: 2006-04-25 08:40
ebayt\eő̃i[nvtcg
excerpt: webayt\eő̃i[nvtcgxvprice : 1,6...
weblog: oꃊt[`imȃƒi[nv̏펯e펯i
tracked: 2006-04-25 08:47
t[i[nvōpoo~҂k@i
excerpt: t[i[nvōpoo~҂k@iyiڍׁzt[i[nvōp...
weblog: oꃊt[`imȃƒi[nv̏펯e펯i
tracked: 2006-04-25 08:49
t[ei[nvkch 2006
excerpt: wt[ei[nvkch 2006x\tgonngcebuprice...
weblog: oꃊt[`imȃƒi[nv̏펯e펯i
tracked: 2006-04-25 08:55
ł yahoo!i[nv
excerpt: wł yahoo!i[nv łxcvxprice : 1,44...
weblog: oꃊt[`imȃƒi[nv̏펯e펯i
tracked: 2006-04-25 09:02
ui[nvnɓv@16\m[xnŗmɉ҂!
------------------------------
ׂővoienjbn
excerpt: wtinŗdtb[o[!!
------------------------------
excerpt: w͂߂ẵt[i[nv\ʒʂ΂ł!!x猽price...
------------------------------
weblog: oꃊt[`imȃƒi[nv̏펯e펯i
tracked: 2006-04-25 10:04
ȓp

Document: http://000sundancer000.livejournal.com/1347789.html
create an account
eskimo kissez - my organic beauty business
recent entries
archive
friends
profile
what my dream guy wants - dannielle's crazy journal
you learn something new every day.
------------------------------
august 2017 12345
6789101112
13141516171819
20212223242526
2728293031
previous | share | next
000sundancer000
07 january 2017 @ 05:19 pm
what my dream guy wants
what does my future husband want in a woman?he wants someone strong, independent, comfortable in her own skin, who knows what she wants.
------------------------------
madly in love with jesus, excited about this gift called life and passionate about her goals and dreams.
------------------------------
she loves people and loves to have fun, singing, dancing, and joking around with a laugh that reverberates off the walls.
------------------------------
i need to find a man who wants this woman and i need to stop falling for men who want someone else.
--

Document: http://0166244.blogspot.com/2014/
and i’m happy to receive your comments about what’s missing from this list.
------------------------------
much of the focus to date has been digital reversioning or incremental improvement of existing processes, in three main areas: information gathering, information parsing/analysis and the distribution of what i would say is just one result of that process – articles in print/content/news broadcasts/online articles/digital news packages in any form – video, written, oral etc..
i’d summarise the 18 this way.
------------------------------
the whole point of “news as a service” (which, yes, will depend on software) is to change the product on offer from pieces of news content to “the state of being well informed.”
i take your point that news may be only one of the products that news companies make when they learn to how to use machines to monitor the environment and detect events.
------------------------------
i am going to do a revised ver